In [27]:
import pandas as pd
import numpy as npy
import xlrd
import re
import hashlib
import sys

base_dir = '/Users/adcxdpf/pset_utils-rajeshkaveti-master/'

def load_words(filename):
    with open(filename, "r") as fd:
        lines = fd.read().splitlines()
    return lines

def load_data(filename):
    data = pd.read_excel(filename,0)
    return data


def load_vectors(filename):
    vector = npy.load(filename)
    return vector

def tokenize(text):
    # Get all "words", including contractions
    # eg tokenize("Hello, I'm Scott") --> ['Hello', "I'm", "Scott"]
    return re.findall("\w[\w']+", text)

def cosine_similarity(a, b):
# 	"""Takes 2 vectors a, b and returns the cosine similarity according 
# 	to the definition of the dot product
# 	"""
# 	dot_product = npy.dot(a, b)
# 	norm_a = npy.linalg.norm(a)
# 	norm_b = npy.linalg.norm(b)
# 	return dot_product / (norm_a * norm_b)


    dot_product = npy.dot(a[0], b.T)
    print(dot_product)
    norm_a = npy.linalg.norm(a)
    norm_b = npy.linalg.norm(b)
    print(norm_a)
    print(norm_b)
    print((norm_a * norm_b))
    return dot_product / (norm_a * norm_b)

def my_distance(vec):
    return 1 - cosine_similarity(vec, my_vec.values[0][0])
 

In [28]:
class WordEmbedding(object):
    
    def __init__(self, words, vector):
       self.words=words
       self.vector=vector

    def __call__(self, word):
        """Embed a word

        :returns: vector, or None if the word is outside of the vocabulary
        :rtype: ndarray
        """
        try:
            #print("Inside call function ***" , word)
            wordIndex=self.words.index(word.lower())
            #print(wordIndex)
            #print(self.vector[wordIndex])
            return self.vector[wordIndex]
        except ValueError:
            return None
        
    
        # Consider how you implement the vocab lookup.  It should be O(1).
        

    @classmethod
    def from_files(cls, word_file, vec_file):
        """Instanciate an embedding from files

        Example::

            embedding = WordEmbedding.from_files('words.txt', 'vecs.npy.gz')

        :rtype: cls
        """
        return cls(load_words(word_file), load_vectors(vec_file))
    
           
    def embed_document(self, text):
        
        words = tokenize(text)
        sent_vec = npy.zeros((1 , 300))
        for w in words:
            try:
                vc = embedding(w)
#                 print(vc[0])
                sent_vec = npy.add(sent_vec, vc) 
#                 print(sent_vec[0][0])
            except:
                pass
        #print(sent_vec)
        return sent_vec

In [29]:
words=load_words(base_dir + "words.txt")
vector = load_vectors(base_dir + "vectors.npy.gz")
data = load_data(base_dir + "hashed.xlsx")
data.set_index('hashed_id')

wordEmbedding = WordEmbedding.from_files(base_dir + "words.txt",base_dir + "vectors.npy.gz")
myvec=wordEmbedding.embed_document("The flushing light actor")
myvec.shape
#myvec

(1, 300)

In [30]:
embedding = WordEmbedding(words,vector)
#a = embedding.embed_document('Get fluent with Python and understand advanced topics.')
a = embedding.embed_document('Get fluent with')
a.ravel().ndim


1

In [31]:
data['learn'][28]

'Get fluent with Python and understand advanced topics.'

In [32]:
a = embedding.embed_document(data['project'][28])
# a

In [33]:
data['learn'] = data['learn'].fillna(value='ZERO')
data['project'] = data['project'].fillna(value='ZERO')


In [34]:
learnvecs = data['learn'].apply(embedding.embed_document) 
#learnvecs = embedding.embed_document(data['learn'][3])
learnvecs = learnvecs.ravel()
learnvecs.shape


(45,)

In [35]:
projvecs = data['project'].apply(embedding.embed_document)
projvecs = projvecs.ravel()
projvecs.shape

(45,)

In [36]:
projvecs.ndim

1

In [37]:
print (learnvecs.shape)
print (projvecs.shape)

(45,)
(45,)


In [38]:
vecs = learnvecs + projvecs

#vecs = npy.c_[learnvecs , projvecs]

#vecs[0].shape

In [39]:
print(type(vecs))

vecs_list = pd.Series(vecs)
# print(type(vecs_list))
# print(vecs_list.ndim)
#print(vecs_list.values)
# print(vecs_list[2].shape)
a=[v for v in vecs[0]]
# a

<class 'numpy.ndarray'>


In [40]:
df = pd.DataFrame(vecs, index=vecs_list.index)
df.columns = ['vectors']
type(df['vectors'])

pandas.core.series.Series

In [41]:
vectors = df

idx = data.index[data['hashed_id']== 'b280302a']
print (idx.values)


my_vec = vectors.loc[idx.values]
my_vec.shape


[28]


(1, 1)

In [42]:
index=data.index[data['hashed_id']=='4cee0953']
print(index)
my_vec = df.iloc[[index.values[0]]]
my_vec_df = pd.DataFrame(my_vec, columns=['vector'])
my_vec.values[0][0]


Int64Index([2], dtype='int64')


array([[ -4.60439120e+00,   2.63486217e+01,  -3.77022124e+01,
          1.07182500e+01,   8.62384032e+00,   2.97469240e+01,
         -7.74944564e+00,  -1.75329659e+01,   1.70392352e+01,
          5.27233112e+02,  -5.70193966e+01,   3.61022014e+00,
          1.99347030e+01,  -9.08192839e+00,  -2.14261649e+01,
          2.73125843e-01,  -2.19200696e+01,   3.53886419e+02,
         -4.54392809e+01,  -2.11332456e+01,  -6.85949741e+00,
         -5.62413281e+00,  -2.69154515e+01,  -3.08587638e+00,
         -3.67116258e+00,   1.75038795e+01,   2.24023984e+00,
         -2.31729398e+01,   1.39284462e+01,  -1.83976360e+01,
         -2.19916281e+01,   1.60392696e+01,   1.87315770e+01,
          1.04661118e+01,   1.07239261e+01,  -1.83931308e+01,
         -6.26317229e+00,   1.74517630e+01,  -2.06584687e+01,
         -2.77358254e+01,  -7.92403809e+00,   2.12217080e+01,
         -1.13202662e+01,  -1.96548839e+01,  -7.37028655e+00,
          3.09344810e+01,  -3.20193152e+00,  -3.22261810e+00,
        

In [43]:
distances = df['vectors'].apply(my_distance)


[ 617683.45914409]
759.86694997
827.461437706
628760.598888
[ 280183.40495]
349.494278806
827.461437706
289193.038411
[ 684692.43089062]
827.461437706
827.461437706
684692.430891
[ 755828.75886521]
933.646255153
827.461437706
772556.272598
[ 1364481.45675451]
1681.32604621
827.461437706
1391232.46745
[ 4551.63532095]
11.4297699387
827.461437706
9457.6938661
[ 315229.81039104]
395.937410908
827.461437706
327622.939271
[ 277741.4806829]
347.783349444
827.461437706
287777.310341
[ 228067.49815642]
286.763143816
827.461437706
237285.443263
[ 347295.76746538]
426.006014546
827.461437706
352503.549268
[ 525332.45522285]
654.712944565
827.461437706
541749.714394
[ 540099.79069957]
669.923964975
827.461437706
554336.247212
[ 411887.20925867]
506.766980516
827.461437706
419330.13428
[ 695271.65464574]
857.148062765
827.461437706
709256.968343
[ 698116.05074173]
860.810549174
827.461437706
712287.534612
[ 24491.75733672]
35.9141622238
827.461437706
29717.5843077
[ 468393.76476264]
582.994527819


In [44]:
type(distances)


pandas.core.series.Series

In [45]:
sortedDistance= distances.sort_values()
farDis = sortedDistance.head(5).index
type(data)
for index in farDis:
    print(data.iloc[index])

hashed_id                                             65c57d2c
learn        I am hoping to learn more about computer scien...
project      I am currently collaborating with a peer to bu...
Name: 4, dtype: object
hashed_id                                             2dacbc07
learn        I'm hoping to get a better understanding of ho...
project      *A Python project completed at a previous comp...
Name: 11, dtype: object
hashed_id                                             19fd3c63
learn        Understand common data science algorithms more...
project      In my PhD work, I built optical instruments to...
Name: 34, dtype: object
hashed_id                                             4cee0953
learn        I'm hoping to improve my Python programming sk...
project      I work in Python and/or R most days as a Data ...
Name: 2, dtype: object
hashed_id                                             93173a73
learn        I think I have alot to learn in this course, b...
project      I helped bu

In [46]:
nearDis= sortedDistance.tail(5).index
for index in nearDis:
    print(data.iloc[index])

hashed_id                                             4585e17d
learn                         Advanced Python for Data Science
project      I worked on deep learning project where we use...
Name: 42, dtype: object
hashed_id                                             322ac11f
learn        Learning how to build more robust Data Science...
project                                                   ZERO
Name: 29, dtype: object
hashed_id                                             bc369f45
learn        Applying data science concepts in building act...
project                                                   ZERO
Name: 17, dtype: object
hashed_id                                      dcd8a5b8
learn        Using python in production and more python
project                                            ZERO
Name: 40, dtype: object
hashed_id    30e95f75
learn            ZERO
project          ZERO
Name: 7, dtype: object


In [47]:
first_elem = vecs[2]
second_elem = my_vec.values[0][0]

#print ("First element : ", type(first_elem) , first_elem)
#print ("Second element : ", type(second_elem) , second_elem)

print(cosine_similarity(first_elem, second_elem))


[ 684692.43089062]
827.461437706
827.461437706
684692.430891
[ 1.]
